# MiniDataChallengeSteering 

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
%matplotlib inline

In [2]:
from astropy.io import fits

In [3]:
# to enlarge the sizes
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10,6),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [4]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [5]:
plt.rcParams.update({'figure.max_open_warning': 0}) # to see a large number of files

In [6]:
%load_ext autoreload
%autoreload 2

# Initialisation

## Python Packages

In [7]:
PATH_LSSTFiltersKG='./LSSTFiltersKG'
PATH_ATMPARAMSIM='./atmparamsim'
PATH_ATMTRANSPSIM='./libradtransim'
PATH_CADENCE='./cadence'
PATH_LSSTPHOTO='./lsstphotometry'
PATH_SED='./pysynphotsed'

In [8]:
sys.path.append(PATH_LSSTFiltersKG)
sys.path.append(PATH_ATMPARAMSIM)
sys.path.append(PATH_ATMTRANSPSIM)
sys.path.append(PATH_CADENCE)
sys.path.append(PATH_LSSTPHOTO)
sys.path.append(PATH_SED)

In [9]:
import libsimulateTranspLSSTScattAbsAer as atm
import libLSSTPhotometry as photo
import libLSSTFiltersKG as lsst
import libpysynphotgridsed as sed

thermtable: /Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs/mtab/tae17277m_tmt.fits
   waveset: Min: 3000.0, Max: 11000.0, Num: None, Delta: 1.0, Log: False
 comptable: /Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs/mtab/16n1832tm_tmc.fits
graphtable: /Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs/mtab/14l1632sm_tmg.fits
      area: 350000.0


## Input file

### Cadence file with atmospheric variation 

- the file has been generated from cadence and atmospheric variation in **GenerateCadenceAtmProgram.ipynb**

In [10]:
cadence_atm_program_file='cadence_field1000_atm.csv'

## Output file

- file with star magnitudes from which one must retrieve atmospheric parameters from the star magnitudes

In [11]:
packet=1
packetsize=50

In [12]:
output_file='cadence_field1000_atm_sim_{}.fits'.format(packet)
NROWMIN=(packet-1)*packetsize
NROWMAX=(packet)*packetsize-1
print 'file=',output_file,' NROWMIN=',NROWMIN,' NROWMAX=',NROWMAX

file= cadence_field1000_atm_sim_1.fits  NROWMIN= 0  NROWMAX= 49


## index

In [13]:
idx_out_num=0
idx_out_night=1
idx_out_expdate=2
idx_out_expmjd=3
idx_out_am=4
idx_out_filt=5
idx_out_vaod=6
idx_out_pwv=7
idx_out_o3=8
idx_out_cld=9
idx_out_res=10

# The atmospheric conditions

In [14]:
df=pd.read_csv(cadence_atm_program_file,index_col=False)

In [15]:
#df=df.iloc[NROWMIN:NROWMAX+1]

In [16]:
df.head()

,Unnamed: 0,Unnamed: 0.1,obshistid,nights,expdate,expmjd,airmass,filter,finseeing,transparency,filtskybrightness,fiveSigmaDepth,rawseeing,humidity,vaod,pwv,o3,cloud
0,0,0,127633,173,14965194,59753.208268,1.370996,g,1.160634,0.25,21.732956,24.241311,0.704040,0.0,0.024183,2.823083,265.663773,46.149395
1,1,1,127670,173,14966692,59753.225609,1.274001,g,1.303756,0.25,21.791891,24.160844,0.859973,0.0,0.050639,4.567981,249.294686,1.183331
2,2,2,136947,189,16353504,59769.276675,1.025141,z,0.695528,0.00,19.037479,23.215870,0.504610,0.0,0.066920,9.802676,292.945586,8.283236
3,3,3,136993,189,16355480,59769.299544,1.012049,z,0.659361,0.00,19.223263,23.364350,0.462390,0.0,0.042499,6.894512,301.130130,1.183331
4,4,4,137990,190,16439213,59770.268672,1.030291,y,0.782293,0.00,17.300000,21.767917,0.633086,0.0,0.056745,3.404716,284.761042,1.183331


In [17]:
df.describe()

,Unnamed: 0,Unnamed: 0.1,obshistid,nights,expdate,expmjd,airmass,finseeing,transparency,filtskybrightness,fiveSigmaDepth,rawseeing,humidity,vaod,pwv,o3,cloud
count,912.000000,912.000000,9.120000e+02,912.000000,9.120000e+02,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.0,912.000000,912.000000,912.000000,912.000000
mean,455.500000,455.500000,1.353989e+06,1996.481360,1.725118e+08,61576.664671,1.216682,0.946423,0.135965,19.480077,23.071331,0.652962,0.0,0.052773,8.041834,271.359450,5.052468
std,263.416021,263.416021,6.749863e+05,1006.667221,8.697478e+07,1006.652540,0.134956,0.205151,0.198053,1.784550,1.057774,0.201697,0.0,0.017962,4.117562,18.568436,10.620855
min,0.000000,0.000000,1.276330e+05,173.000000,1.496519e+07,59753.208268,1.010873,0.550999,0.000000,16.332019,20.909705,0.212684,0.0,0.024183,1.078185,224.741054,1.183331
25%,227.750000,227.750000,8.365668e+05,1223.000000,1.056970e+08,60803.345134,1.095562,0.786528,0.000000,17.300000,22.118978,0.506298,0.0,0.040464,5.149614,257.479229,1.183331
50%,455.500000,455.500000,1.387090e+06,2030.500000,1.754551e+08,61610.730403,1.212443,0.906078,0.000000,19.965937,23.207572,0.618785,0.0,0.050639,6.894512,268.391955,1.183331
75%,683.250000,683.250000,1.897182e+06,2798.000000,2.417535e+08,62378.073410,1.342497,1.080563,0.250000,21.104146,23.995230,0.769933,0.0,0.060815,9.802676,284.761042,3.549966
max,911.000000,911.000000,2.434258e+06,3627.000000,3.133765e+08,63207.043279,1.503813,1.501938,0.625000,22.237331,24.919995,1.456743,0.0,0.123904,28.996557,352.965575,110.048538


In [18]:
NbAtmSimul=len(df)

## Atmospheric simulation

In [19]:
all_wl=[]
all_atmtran=[]

In [20]:
#NbAtmSimul=10
lambda0_aerosol=500. # nm

In [ ]:
photo_atm=photo.Atmosphere('libradtran')

In [ ]:
index=0
while index<NbAtmSimul:
    am=df['airmass'].iloc[index]
    pwv=df['pwv'].iloc[index]
    ozone=df['o3'].iloc[index]
    tau_aerosol=df['vaod'].iloc[index]
    print '**************************************************************************************************'
    print '*  atmospheric simulation with libradtran : idx=',index,' am=',am,' vaod=',tau_aerosol,' pwv=',pwv,' O3=',ozone ,' *'
    '********************************************************************************************************'
    path,thefile=atm.ProcessSimulationaer(am,pwv,ozone,lambda0_aerosol,tau_aerosol) 
    fullfilename=os.path.join(path,thefile)
    atm_data=np.loadtxt(fullfilename)
    wl_atm=atm_data[:,0]
    tr_atm=atm_data[:,1]  
    all_wl.append(wl_atm)
    all_atmtran.append(tr_atm) 
    if index==0:
        nbcols=len(wl_atm)
        nbrows=NbAtmSimul+1
        data_atm=np.zeros((nbrows,nbcols))
        data_atm[0,:]=wl_atm
    index+=1
    data_atm[index,:]=tr_atm

**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 0  am= 1.370996  vaod= 0.0241827311217  pwv= 2.82308303298  O3= 265.663773362  *
--------------------------------------------
 1) airmass =  1.370996
 2) pwv =  2.82308303298
 3) oz =  265.663773362
 4) wl0 =  500.0
 5) tau0 =  0.0241827311217
--------------------------------------------
!!!!!! file  simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z13_wv28_oz26_aer2.OUT already exist : skip libradtran simulation
**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 1  am= 1.274001  vaod= 0.0506393918768  pwv= 4.56798130205  O3= 249.294685593  *
--------------------------------------------
 1) airmass =  1.274001
 2) pwv =  4.56798130205
 3) oz =  249.294685593
 4) wl0 =  500.0
 5) tau0 =  0.0506393918768
----------------------------------

**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 11  am= 1.051037  vaod= 0.0547096473775  pwv= 15.0373709165  O3= 262.935592067  *
--------------------------------------------
 1) airmass =  1.051037
 2) pwv =  15.0373709165
 3) oz =  262.935592067
 4) wl0 =  500.0
 5) tau0 =  0.0547096473775
--------------------------------------------
('Running uvspec with input file: ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z10_wv150_oz26_aer5.INP')
('Output to file                : ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z10_wv150_oz26_aer5.OUT')
('uvspec cmd: ', '/Users/dagoret/MacOSX/External/libRadtran/libRadtran-2.0.1/bin/uvspec  < simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z10_wv150_oz26_aer5.INP > simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z10_wv150_oz26_aer5.OUT')
*************************************************************

**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 21  am= 1.021651  vaod= 0.0648852861295  pwv= 12.7108398911  O3= 262.935592067  *
--------------------------------------------
 1) airmass =  1.021651
 2) pwv =  12.7108398911
 3) oz =  262.935592067
 4) wl0 =  500.0
 5) tau0 =  0.0648852861295
--------------------------------------------
('Running uvspec with input file: ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z10_wv127_oz26_aer6.INP')
('Output to file                : ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z10_wv127_oz26_aer6.OUT')
('uvspec cmd: ', '/Users/dagoret/MacOSX/External/libRadtran/libRadtran-2.0.1/bin/uvspec  < simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z10_wv127_oz26_aer6.INP > simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z10_wv127_oz26_aer6.OUT')
*************************************************************

**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 32  am= 1.058005  vaod= 0.0343583698736  pwv= 15.0373709165  O3= 254.751048183  *
--------------------------------------------
 1) airmass =  1.058005
 2) pwv =  15.0373709165
 3) oz =  254.751048183
 4) wl0 =  500.0
 5) tau0 =  0.0343583698736
--------------------------------------------
('Running uvspec with input file: ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z10_wv150_oz25_aer3.INP')
('Output to file                : ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z10_wv150_oz25_aer3.OUT')
('uvspec cmd: ', '/Users/dagoret/MacOSX/External/libRadtran/libRadtran-2.0.1/bin/uvspec  < simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z10_wv150_oz25_aer3.INP > simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z10_wv150_oz25_aer3.OUT')
*************************************************************

**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 42  am= 1.110054  vaod= 0.0486042641264  pwv= 3.40471578933  O3= 271.120135952  *
--------------------------------------------
 1) airmass =  1.110054
 2) pwv =  3.40471578933
 3) oz =  271.120135952
 4) wl0 =  500.0
 5) tau0 =  0.0486042641264
--------------------------------------------
('Running uvspec with input file: ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z11_wv34_oz27_aer4.INP')
('Output to file                : ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z11_wv34_oz27_aer4.OUT')
('uvspec cmd: ', '/Users/dagoret/MacOSX/External/libRadtran/libRadtran-2.0.1/bin/uvspec  < simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z11_wv34_oz27_aer4.INP > simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z11_wv34_oz27_aer4.OUT')
*****************************************************************

...

**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 359  am= 1.011031  vaod= 0.0486042641264  pwv= 9.80267610927  O3= 284.761042427  *
--------------------------------------------
 1) airmass =  1.011031
 2) pwv =  9.80267610927
 3) oz =  284.761042427
 4) wl0 =  500.0
 5) tau0 =  0.0486042641264
--------------------------------------------
('Running uvspec with input file: ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z10_wv98_oz28_aer4.INP')
('Output to file                : ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z10_wv98_oz28_aer4.OUT')
('uvspec cmd: ', '/Users/dagoret/MacOSX/External/libRadtran/libRadtran-2.0.1/bin/uvspec  < simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z10_wv98_oz28_aer4.INP > simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z10_wv98_oz28_aer4.OUT')
****************************************************************

**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 370  am= 1.351349  vaod= 0.0913419468846  pwv= 5.14961405841  O3= 257.479229477  *
--------------------------------------------
 1) airmass =  1.351349
 2) pwv =  5.14961405841
 3) oz =  257.479229477
 4) wl0 =  500.0
 5) tau0 =  0.0913419468846
--------------------------------------------
('Running uvspec with input file: ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z13_wv51_oz25_aer9.INP')
('Output to file                : ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z13_wv51_oz25_aer9.OUT')
('uvspec cmd: ', '/Users/dagoret/MacOSX/External/libRadtran/libRadtran-2.0.1/bin/uvspec  < simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z13_wv51_oz25_aer9.INP > simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z13_wv51_oz25_aer9.OUT')
****************************************************************

**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 383  am= 1.103166  vaod= 0.0567447751279  pwv= 14.4557381601  O3= 260.207410772  *
--------------------------------------------
 1) airmass =  1.103166
 2) pwv =  14.4557381601
 3) oz =  260.207410772
 4) wl0 =  500.0
 5) tau0 =  0.0567447751279
--------------------------------------------
('Running uvspec with input file: ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z11_wv144_oz26_aer5.INP')
('Output to file                : ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z11_wv144_oz26_aer5.OUT')
('uvspec cmd: ', '/Users/dagoret/MacOSX/External/libRadtran/libRadtran-2.0.1/bin/uvspec  < simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z11_wv144_oz26_aer5.INP > simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z11_wv144_oz26_aer5.OUT')
************************************************************

**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 394  am= 1.011982  vaod= 0.036393497624  pwv= 6.89451232748  O3= 295.673767606  *
--------------------------------------------
 1) airmass =  1.011982
 2) pwv =  6.89451232748
 3) oz =  295.673767606
 4) wl0 =  500.0
 5) tau0 =  0.036393497624
--------------------------------------------
!!!!!! file  simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z10_wv68_oz29_aer3.OUT already exist : skip libradtran simulation
**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 395  am= 1.014443  vaod= 0.0526745196272  pwv= 9.80267610927  O3= 301.130130196  *
--------------------------------------------
 1) airmass =  1.014443
 2) pwv =  9.80267610927
 3) oz =  301.130130196
 4) wl0 =  500.0
 5) tau0 =  0.0526745196272
--------------------------------

**************************************************************************************************
*  atmospheric simulation with libradtran : idx= 405  am= 1.216705  vaod= 0.0567447751279  pwv= 8.05777784019  O3= 257.479229477  *
--------------------------------------------
 1) airmass =  1.216705
 2) pwv =  8.05777784019
 3) oz =  257.479229477
 4) wl0 =  500.0
 5) tau0 =  0.0567447751279
--------------------------------------------
('Running uvspec with input file: ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z12_wv80_oz25_aer5.INP')
('Output to file                : ', 'simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z12_wv80_oz25_aer5.OUT')
('uvspec cmd: ', '/Users/dagoret/MacOSX/External/libRadtran/libRadtran-2.0.1/bin/uvspec  < simulations/RT/2.0.1/LS/pp/us/sa/rt/in/RT_LS_pp_us_sa_rt_z12_wv80_oz25_aer5.INP > simulations/RT/2.0.1/LS/pp/us/sa/rt/out/RT_LS_pp_us_sa_rt_z12_wv80_oz25_aer5.OUT')
****************************************************************

In [ ]:
all_wl=np.array(all_wl)
all_atmtran=np.array(all_atmtran)

In [ ]:
all_wl.shape

In [ ]:
all_atmtran.shape

In [ ]:
data_atm.shape

In [ ]:
for idx in np.arange(1,NbAtmSimul+1):
    plt.plot(data_atm[0],data_atm[idx])
plt.grid(True)
plt.xlabel("$\lambda$ (nm)")
plt.title("Atmospheric transparency")

# output file

In [ ]:
NBROW=NbAtmSimul+1
NBCOL=idx_out_res+nbcols

In [ ]:
data_out=np.zeros((NBROW,NBCOL))

In [ ]:
data_out[0,idx_out_res:]=data_atm[0]
for idx in np.arange(1,NBROW):
    data_out[idx,idx_out_num]=idx-1
    data_out[idx,idx_out_night]=df['nights'].iloc[idx-1]
    data_out[idx,idx_out_expdate]=df['expdate'].iloc[idx-1]
    data_out[idx,idx_out_expmjd]=df['expmjd'].iloc[idx-1]
    data_out[idx,idx_out_am]=df['airmass'].iloc[idx-1]
    filtnum=photo.band_to_number_2[df['filter'].iloc[idx-1]] # convert filter into number
    data_out[idx,idx_out_filt]=filtnum
    data_out[idx,idx_out_vaod]=df['vaod'].iloc[idx-1]
    data_out[idx,idx_out_pwv]=df['pwv'].iloc[idx-1]
    data_out[idx,idx_out_o3]=df['o3'].iloc[idx-1]
    data_out[idx,idx_out_cld]=df['cloud'].iloc[idx-1]
    data_out[idx,idx_out_res:]=data_atm[idx]


In [ ]:
plt.imshow(data_out[1:,idx_out_res:],cmap='jet')

In [ ]:
hdr = fits.Header()
hdr['NBATMSIM']=NbAtmSimul
hdr['ID_NUM']=idx_out_num
hdr['ID_NIGHT']=idx_out_night
hdr['ID_DATE']=idx_out_expdate
hdr['ID_EXMJD']=idx_out_expmjd
hdr['ID_AM']=idx_out_am
hdr['ID_FILT']=idx_out_filt
hdr['ID_VAOD']=idx_out_vaod
hdr['ID_PWV']=idx_out_pwv
hdr['ID_O3']=idx_out_o3
hdr['ID_CLD']=idx_out_cld
hdr['ID_RES']=idx_out_res

hdu = fits.PrimaryHDU(data_out,header=hdr)
hdu.writeto(output_file,overwrite=True)

In [ ]:
data_out.shape

In [ ]:
for idx in np.arange(1,NbAtmSimul+1):
    plt.plot(data_out[0,idx_out_res:],data_out[idx,idx_out_res:])
#   plt.plot(data_out[0,idx_out_res:],data_out[912,idx_out_res:])
plt.grid(True)
plt.title("Atmospheric transparency")
plt.xlabel("$\lambda$ (nm)")

In [ ]:
plt.hist(data_out[1:,idx_out_am],bins=50);
plt.grid(True)
plt.xlabel("airmass")